In [3]:
import wandb
import os
from omegaconf import OmegaConf


In [ ]:
ENTITY = "floly"               
PROJECT = "effortsim-optuna"   
METRIC = "best/avg_reward_100" 
TOPK = 5                       


In [5]:
api = wandb.Api()
runs = api.runs(f"{ENTITY}/{PROJECT}")

def get_score(r):
    return r.summary.get(METRIC, float("-inf"))

valid_runs = [r for r in runs if get_score(r) != float("-inf")]
valid_runs.sort(key=lambda r: get_score(r), reverse=True)

if not valid_runs:
    raise ValueError(f"No valid runs found in project `{PROJECT}` with metric `{METRIC}`.")


In [6]:
print(f"\n🏆 Top {min(TOPK, len(valid_runs))} runs sorted by `{METRIC}`:\n")

for r in valid_runs[:TOPK]:
    cfg = r.config
    score = get_score(r)
    print(f"{r.name or r.id:30s}  score={score:8.3f}  "
          f"lr={cfg.get('learning_rate')}  clip={cfg.get('eps_clip')}  "
          f"gamma={cfg.get('gamma')}  gae={cfg.get('gae_lambda')}  "
          f"mb={cfg.get('mini_batch_size')}  epochs={cfg.get('ppo_epochs')}")



🏆 Top 5 runs sorted by `best/avg_reward_100`:

trial-30-9cba10df               score= 209.561  lr=0.00030220863975091326  clip=0.3366362052213431  gamma=0.9803970348868488  gae=0.9655754964372536  mb=64  epochs=9
trial-37-8594292f               score= 199.201  lr=0.0028419306534024223  clip=0.17586948516313322  gamma=0.978576170113718  gae=0.9642503000446536  mb=256  epochs=14
trial-19-5cba18c6               score= 123.356  lr=0.0010837116643555484  clip=0.3383562707679318  gamma=0.971355736958336  gae=0.917290935380037  mb=128  epochs=12
trial-29-5a6399ee               score=  98.956  lr=0.002159372797460324  clip=0.2702897824016524  gamma=0.9834992587461344  gae=0.930900289953142  mb=128  epochs=13
trial-39-690d9172               score=  85.186  lr=0.0009070212905880988  clip=0.25347866134968033  gamma=0.976174298303572  gae=0.9152871506676956  mb=64  epochs=10


In [7]:
# Pick best run (index 0)
best_run = valid_runs[0]
best_cfg = OmegaConf.create(best_run.config)

# Save config to disk
os.makedirs("best", exist_ok=True)
out_path = f"best/{best_run.name or best_run.id}.yaml"
OmegaConf.save(best_cfg, out_path)

print(f"\n✅ Best config saved to: {out_path}")



✅ Best config saved to: best/trial-30-9cba10df.yaml
